In [196]:
'''
from scipy.io import loadmat
annots = loadmat('Data/IIIT5K/traindata.mat')
annots['traindata'][0][100][1][0]
# second para: row . Extract name
from __future__ import print_function
from __future__ import division
'''

import os.path
import argparse
import random
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import numpy as np
#from warpctc_pytorch import CTCLoss
import os
#import dataset
from dataset import lmdbDataset, resizeNormalize, randomSequentialSampler, alignCollate
import create_dataset


lmdb_path = "Data/lmdb"
train_dataset = lmdbDataset(root=lmdb_path)  # root to dataset
assert train_dataset

#sampler = randomSequentialSampler(train_dataset, 100)  # batch size = 100
sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=100,
    shuffle=True, sampler=sampler,
    num_workers=4,
    collate_fn=alignCollate(imgH=32, imgW=100))


In [197]:
train_iter = iter(train_loader)

In [198]:
images, texts = train_iter.next()
images2, texts2 = train_iter.next()

In [199]:
# texts is tuple, each element is a string label. 100 in total
import collections
print(isinstance(texts, collections.Iterable))
new_texts = [s[2:-1] for s in texts]
print(new_texts)
print(texts)

True
['THEYRE', 'SPACE', 'BANSAL', 'LANZMANN', 'EXCLUSIVE', 'AND', '1', '23', 'KAUSHAL', 'HAMMER', 'SAVE', 'WWWWEBSCOM', 'KNIGHT', 'RESULTS', 'TAYLOR', 'STATE', 'SIP', 'LANE', 'FUTURE', 'FROM', 'CRESTDOWN', 'REMITTANCES', 'THE', 'FAX', 'RECRUITMENT', '37', '12530', 'FANTASTISK', 'DR', 'INDIA', 'GC', 'FERRABY', 'DOGS', 'EVERY', 'PERSEEL', 'YOUR', 'VILLA', 'CAMPAIGN', 'FARM', 'ALL', '5', 'FOX', 'BANK', 'HOFFMAN', 'AS', 'MART', 'ROZOWEJ', 'SDN', 'ZEROEMISSIONS', 'GREM', 'FOR', 'DEPOSITPHOTOS', '1300', 'WILT', 'CAREER', '360', 'COPIES', 'COPY', 'OF', 'TRINITYLEEDS', 'EN', 'UP', 'ONGC', 'DANGERS', 'CAUTION', 'REPORT', 'CANNIESBURN', 'NEW', 'MADE', 'MEDICAL', 'KIRKLISTON', 'PERSONAL', 'LIVING', 'FRANKLIN', 'HOW', '8401950', 'END', 'ANDRIEJ', 'DO', 'KEENER', 'THEM', 'NAME', 'LANE', 'TRACK', 'CHRYSLER', 'SCHWARZENEGGER', 'ONE', 'ALL', '1306', 'BRASSERIE', 'OR', 'WHEN', 'ROADS', 'INDIA', 'THROUGHOUT', 'DO', '8', '2', 'DAY', '137']
("b'THEYRE'", "b'SPACE'", "b'BANSAL'", "b'LANZMANN'", "b'EXCLUSI

In [200]:
# check batch image size
print(images.shape)


torch.Size([100, 1, 32, 100])


# convert image and label to Torch tensor

In [201]:
# Refine all image to wanted size: (32,100)
# return v
def refine_img(v, img):
    v.data.resize_(img.size()).copy_(img)

In [202]:
# store batch image and text in Tensor
image = torch.FloatTensor(100, 1, 32, 100)  # 100: batch size  imgH, imgW
text = torch.IntTensor(100 * 5)
batch_size = torch.IntTensor(100)

image = Variable(image)
text = Variable(text)
batch_size = Variable(batch_size)

# TEST
refine_img(image, images)  # images: read from train_iterator
#print(images)


In [203]:
# Now we want to encode and decode the label
# Example:
# Encode: 7abc --> [7,10,11,12]  (IntTensor)
# Docode: [11,12,13] --> [b,c,d]
# *****TODO*****
# wrap all functions in a CLASS

# Define alphabet in config.py
alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'

class LabelConverter(object):
    def __init__(self, alphabet, ignore_case = True):
        self.ignore_case = ignore_case
        if self.ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'
        self.dict = {}
        for index, c in enumerate(self.alphabet):
            self.dict[c] = index + 1  # index 0 reserved for blank
    
    def encode(self, labels):
        # input: labels: list of labels name
        # return: [labels encoded], [labels length]
        length = [len(c) for c in labels]
        text = ''.join(labels)
        encoded = []
        for c in text:
            encoded.append(self.dict[c.lower()] )
        return torch.IntTensor(encoded), torch.IntTensor(length)
    
    def decode(self, text, length):
        # decode to strs, batch mode
        # text: list of encodings    length: list of length
        assert sum(length) == text.numel()  # .numel used to calculate number of elements in text
        decodings = []
        pos = 0
        for str_len in length:
            encode = text[pos:pos+str_len]
            decode = ''
            for digit in encode:
                decode += self.alphabet[digit-1]
            decodings.append(decode)
            pos+= str_len
        return decodings



In [218]:
# TEST

labelConverter = LabelConverter(alphabet)
encode_label, encode_length = labelConverter.encode(new_texts)
encode_label.shape



torch.Size([530])

#  Manage Loss and Optimizer

# new a network object and initialize its weights

In [214]:
# Class RCNN definition
import torch
import torch.nn as nn

class BinLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BinLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output

class CRNN(nn.Module):
    def __init__(self, imgH, imgW, n_hidden = 2, n_class=36, init_weights=True):
        super(CRNN, self).__init__()
        self.cfg = [64, 'M22', 128, 'M22', 256, 256, 'M12', 512, 'bc', 512, 'bc', 'M12', 512]
        self.out_channels = [64, 128, 256, 256, 512, 512, 512]  # output channels of each conv layers
        self.layers = self.make_layers(self.cfg)
        self.rnn = nn.Sequential(
            BinLSTM(512, n_hidden, n_hidden),
            BinLSTM(n_hidden, n_hidden, n_class))
        
    def make_layers(self, cfg):
        layers = []
        in_channels = 1  # in this example, channel is 1
        layer_count = -1  # count conv layers
        for v in cfg:
            if v == 'M22':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            elif v == 'M12':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,1), padding=(0,1))]
            elif v == 'bc':
                layers += [nn.BatchNorm2d(self.out_channels[layer_count]), nn.ReLU(inplace=True)]
            else:
                if layer_count == 5:
                    conv2d = nn.Conv2d(in_channels, v, kernel_size=2, padding=0)
                else:
                    conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                layers += [conv2d, nn.ReLU(inplace=True)]
                layer_count += 1
                in_channels = v
        return nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.layers(x)
        b,c,h,w = x.size()
        # print(b,c,h,w)  100,512,1,26
        # convert it to   26,100,512
        assert h == 1, "the height of conv must be 1"
        x = x.squeeze(2) # squeeze second dimension --> 100 512 26
        x = x.permute(2, 0, 1)  # 26 100 512
        # input to the lstm MUST be 3D: 
        # The first axis is the sequence itself, the second indexes mini-batch size
        x = self.rnn(x)

        return x


In [215]:
# TEST network
crnn = CRNN(32, 100)
output = crnn(images)

In [216]:
output.shape

torch.Size([26, 100, 36])

In [217]:


# TEST network
test_tensor = torch.randn(100, 1, 32, 100)

texts.shape


AttributeError: 'tuple' object has no attribute 'shape'